In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from transformers import TFViTForImageClassification, ViTFeatureExtractor, ViTImageProcessor
from datasets import load_dataset
import datasets
import os
import datasets
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')
os.mkdir("/content/drive/MyDrive/EE992/ViT/results/")
path = "/content/drive/MyDrive/EE992/ViT/results/"

Mounted at /content/drive


In [5]:
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = TFViTForImageClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# from transformers import ViTImageProcessor

# Load CIFAR-100 dataset
dataset = datasets.load_dataset("cifar100")
img_class_labels = dataset['train'].features['fine_label'].names
# Rename column to labels for .to_tf_dataset later
dataset = dataset.rename_column("fine_label", "labels")

# Initialize the ViT image processor for 224x224 resizing and normalization
feature_extractor = ViTImageProcessor(
    size=224,
    do_resize=True,
    do_normalize=True,
    image_mean=[0.5, 0.5, 0.5],
    image_std=[0.5, 0.5, 0.5],
)

# Create a preprocessing function
def preprocess_function(examples):
    images = examples["img"]
    # Process images using ViTImageProcessor
    inputs = feature_extractor(images=images, return_tensors="tf")
    inputs["labels"] = examples["labels"]
    return inputs

# Apply preprocessing
processed_dataset = dataset.map(preprocess_function, batched=True)

# Split into train/validation sets
test_size = 0.01
train_val_split = processed_dataset["train"].train_test_split(test_size=test_size)

# Create the final dataset dictionary
final_dataset = datasets.DatasetDict({
    "train": train_val_split["train"],
    "validation": train_val_split["test"],
    "test": processed_dataset["test"]  # Use the original test set
})

# Convert to TensorFlow datasets
tf_train_dataset = final_dataset["train"].to_tf_dataset(
    columns=["pixel_values", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=None,
)

tf_validation_dataset = final_dataset["validation"].to_tf_dataset(
    columns=["pixel_values", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=None,
)

tf_test_dataset = final_dataset["test"].to_tf_dataset(
    columns=["pixel_values", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=None,
)

README.md:   0%|          | 0.00/9.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/119M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
#get all numeric labels from the test set
labels_test = final_dataset['test']['labels']


In [8]:
def calculate_metrics(true_labels, predicted_labels):
    #calculate precision, recall, and F1 score
    precision = precision_score(true_labels, predicted_labels, average='macro')
    recall = recall_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')
    acc = accuracy_score(true_labels, predicted_labels)

    return precision, recall, f1, acc


In [9]:
import pandas as pd

lr = [5e-5, 1e-5, 1e-6]
decay = [0.04, 0.1, 0.3]


for LR in lr:

    for DECAY in decay:

        model_csv = "ViT_"+str(LR) + '_' + str(DECAY)

        model_name = "google/vit-base-patch16-224-in21k"

        id2label = {str(i): label for i, label in enumerate(img_class_labels)}
        label2id = {v: k for k, v in id2label.items()}


        # load pre-trained ViT model
        model = TFViTForImageClassification.from_pretrained(
            model_name,
            num_labels=len(img_class_labels),
            id2label=id2label,
            label2id=label2id,
        )

        # Define optimizer and loss
        optimizer = tf.keras.optimizers.AdamW(learning_rate=LR, weight_decay = DECAY)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

        # Compile the model
        model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

        # Train the model
        history = model.fit(
            tf_train_dataset,
            validation_data=tf_validation_dataset,
            epochs=5,
            callbacks=[early_stopping]
        )

        y_hat = model.predict(tf_test_dataset)

        predicted_classes = np.argmax(y_hat[1], axis=1)

        precision, recall, f1, acc = calculate_metrics(labels_test, predicted_classes)

        # Print the results
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1}")
        print(f"Accuracy: {acc}")

        # convert history to DataFrame
        history_df = pd.DataFrame(history.history)

        history_df.to_csv(path+model_csv+".csv", index=False)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
3094/3094 [==============================] - 384s 110ms/step - loss: 1.3712 - accuracy: 0.8199 - val_loss: 0.5046 - val_accuracy: 0.8720
Epoch 2/5
3094/3094 [==============================] - 337s 109ms/step - loss: 0.2665 - accuracy: 0.9289 - val_loss: 0.4142 - val_accuracy: 0.8920
Epoch 3/5
3094/3094 [==============================] - 337s 109ms/step - loss: 0.1569 - accuracy: 0.9574 - val_loss: 0.4863 - val_accuracy: 0.8680
Epoch 4/5
625/625 [==============================] - 67s 103ms/step
Precision: 0.8984379092026026
Recall: 0.8935
F1 Score: 0.8935648183317579
Accuracy: 0.8935


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
3094/3094 [==============================] - 376s 109ms/step - loss: 1.3705 - accuracy: 0.8203 - val_loss: 0.5135 - val_accuracy: 0.8660
Epoch 2/5
3094/3094 [==============================] - 337s 109ms/step - loss: 0.2629 - accuracy: 0.9320 - val_loss: 0.4126 - val_accuracy: 0.8880
Epoch 3/5
3094/3094 [==============================] - 337s 109ms/step - loss: 0.1651 - accuracy: 0.9542 - val_loss: 0.4367 - val_accuracy: 0.8800
Epoch 4/5
625/625 [==============================] - 67s 102ms/step
Precision: 0.8984894460656894
Recall: 0.8932999999999999
F1 Score: 0.893490415791963
Accuracy: 0.8933


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1/5
3094/3094 [==============================] - 376s 109ms/step - loss: 4.5123 - accuracy: 0.1320 - val_loss: 4.4047 - val_accuracy: 0.3800
Epoch 2/5
3094/3094 [==============================] - 337s 109ms/step - loss: 4.2701 - accuracy: 0.5891 - val_loss: 4.1436 - val_accuracy: 0.7120
Epoch 3/5
3094/3094 [==============================] - 336s 109ms/step - loss: 3.9898 - accuracy: 0.7656 - val_loss: 3.8651 - val_accuracy: 0.7700
Epoch 4/5
3094/3094 [==============================] - 334s 108ms/step - loss: 3.7040 - accuracy: 0.8047 - val_loss: 3.5944 - val_accuracy: 0.7860
Epoch 5/5
625/625 [==============================] - 67s 102ms/step
Precision: 0.8478729883100228
Recall: 0.8249000000000001
F1 Score: 0.8208111601012574
Accuracy: 0.8249


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
3094/3094 [==============================] - 376s 109ms/step - loss: 4.5214 - accuracy: 0.1124 - val_loss: 4.4163 - val_accuracy: 0.3240
Epoch 2/5
3094/3094 [==============================] - 339s 110ms/step - loss: 4.2799 - accuracy: 0.5618 - val_loss: 4.1488 - val_accuracy: 0.6820
Epoch 3/5
3094/3094 [==============================] - 335s 108ms/step - loss: 3.9939 - accuracy: 0.7550 - val_loss: 3.8594 - val_accuracy: 0.7660
Epoch 4/5
3094/3094 [==============================] - 335s 108ms/step - loss: 3.7055 - accuracy: 0.8092 - val_loss: 3.5856 - val_accuracy: 0.7940
Epoch 5/5
625/625 [==============================] - 67s 102ms/step
Precision: 0.8410494854142212
Recall: 0.8285000000000001
F1 Score: 0.8251072975818446
Accuracy: 0.8285


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
3094/3094 [==============================] - 376s 109ms/step - loss: 4.5200 - accuracy: 0.1136 - val_loss: 4.4108 - val_accuracy: 0.3260
Epoch 2/5
3094/3094 [==============================] - 335s 108ms/step - loss: 4.2766 - accuracy: 0.5796 - val_loss: 4.1442 - val_accuracy: 0.7160
Epoch 3/5
3094/3094 [==============================] - 337s 109ms/step - loss: 3.9923 - accuracy: 0.7615 - val_loss: 3.8578 - val_accuracy: 0.7700
Epoch 4/5
3094/3094 [==============================] - 335s 108ms/step - loss: 3.7051 - accuracy: 0.8070 - val_loss: 3.5841 - val_accuracy: 0.7860
Epoch 5/5
625/625 [==============================] - 67s 102ms/step
Precision: 0.8387431507069728
Recall: 0.823
F1 Score: 0.817593946548423
Accuracy: 0.823
